<a href="https://colab.research.google.com/github/PizdecAnuNahui/Portfolio-Dashboard/blob/main/Project_2_Backtesting_a_Forecast_Based_Trading_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# DCF model

import yfinance as yf
import numpy as np
import pandas as pd

ticker = "AAPL"
stock = yf.Ticker(ticker)

# Get cashflow dataframe and extract Free Cash Flow
cashflow = stock.cashflow
try:
    fcf = cashflow.loc["Free Cash Flow"].dropna().astype(float)[::-1]  # reverse for oldest -> newest
except KeyError:
    print("❌ 'Free Cash Flow' not found in cashflow data.")
    print("Available fields:\n", cashflow.index.tolist())
    raise

# Forecast future FCFs using average growth
avg_growth = fcf.pct_change().mean()
last_fcf = fcf[-1]
years = 5
forecast_fcfs = [last_fcf * ((1 + avg_growth) ** i) for i in range(1, years + 1)]

# DCF assumptions
wacc = 0.08
terminal_growth = 0.02
terminal_value = forecast_fcfs[-1] * (1 + terminal_growth) / (wacc - terminal_growth)

# Discount forecasted FCFs and terminal value
discounted_fcfs = [fcf / ((1 + wacc) ** i) for i, fcf in enumerate(forecast_fcfs, 1)]
discounted_terminal = terminal_value / ((1 + wacc) ** years)

enterprise_value = sum(discounted_fcfs) + discounted_terminal
enterprise_value_b = enterprise_value / 1e9

# Market cap comparison
market_cap = stock.info['marketCap'] / 1e9

# Output Summary
summary = {
    "Ticker": ticker,
    "Last FCF (B)": round(last_fcf / 1e9, 2),
    "Avg FCF Growth": f"{round(avg_growth * 100, 2)}%",
    "WACC": f"{round(wacc * 100, 2)}%",
    "Enterprise Value (DCF)": round(enterprise_value_b, 2),
    "Market Cap (B)": round(market_cap, 2),
    "DCF Implied Upside (%)": round((enterprise_value_b - market_cap) / market_cap * 100, 2)
}

summary_df = pd.DataFrame([summary])
print("📊 DCF Valuation Summary:")
print(summary_df)


<ipython-input-6-097e432fadf8>:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  last_fcf = fcf[-1]


📊 DCF Valuation Summary:
  Ticker  Last FCF (B) Avg FCF Growth  WACC  Enterprise Value (DCF)  \
0   AAPL        108.81          6.17%  8.0%                 2215.29   

   Market Cap (B)  DCF Implied Upside (%)  
0         3315.83                  -33.19  


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# List of 10 stocks
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "NVDA", "JNJ", "V", "PG"]

# Assumptions
wacc = 0.08
years = 5
perpetual_growth_rate = 0.025

summary = []

for ticker in tickers:
    try:
        print(f"🔄 Processing {ticker}...")
        stock = yf.Ticker(ticker)

        cashflow = stock.cashflow
        if cashflow is None or cashflow.empty:
            print(f"❌ No cashflow data for {ticker}")
            continue

        cashflow = cashflow.T
        if 'Total Cash From Operating Activities' not in cashflow.columns or 'Capital Expenditures' not in cashflow.columns:
            print(f"❌ Required fields not found in cashflow for {ticker}")
            continue

        fcf = (cashflow['Total Cash From Operating Activities'] - cashflow['Capital Expenditures']) / 1e9
        fcf = fcf[fcf.notna()]
        if len(fcf) < 2:
            print(f"❌ Not enough FCF history for {ticker}")
            continue

        last_fcf = fcf[-1]
        avg_growth = fcf.pct_change().dropna().mean()

        # Forecast future FCFs
        forecasted_fcfs = [last_fcf * ((1 + avg_growth) ** i) for i in range(1, years + 1)]
        discounted_fcfs = [fcf / ((1 + wacc) ** i) for i, fcf in enumerate(forecasted_fcfs, 1)]
        terminal_value = (forecasted_fcfs[-1] * (1 + perpetual_growth_rate)) / (wacc - perpetual_growth_rate)
        discounted_terminal = terminal_value / ((1 + wacc) ** years)

        enterprise_value = sum(discounted_fcfs) + discounted_terminal

        # Market Cap
        market_cap = stock.info.get("marketCap", np.nan) / 1e9
        if np.isnan(market_cap):
            print(f"❌ Market cap not available for {ticker}")
            continue

        implied_upside = round((enterprise_value - market_cap) / market_cap * 100, 2)

        summary.append({
            "Ticker": ticker,
            "Last FCF (B)": round(last_fcf, 2),
            "Avg FCF Growth": f"{round(avg_growth * 100, 2)}%",
            "WACC": f"{round(wacc * 100, 1)}%",
            "Enterprise Value (DCF)": round(enterprise_value, 2),
            "Market Cap (B)": round(market_cap, 2),
            "DCF Implied Upside (%)": implied_upside
        })
    except Exception as e:
        print(f"⚠️ Error processing {ticker}: {e}")

# Display the summary table
summary_df = pd.DataFrame(summary)
import ace_tools as tools; tools.display_dataframe_to_user(name="DCF Valuation Summary for 10 Stocks", dataframe=summary_df)


🔄 Processing AAPL...
❌ Required fields not found in cashflow for AAPL
🔄 Processing MSFT...
❌ Required fields not found in cashflow for MSFT
🔄 Processing GOOGL...
❌ Required fields not found in cashflow for GOOGL
🔄 Processing AMZN...
❌ Required fields not found in cashflow for AMZN
🔄 Processing META...
❌ Required fields not found in cashflow for META
🔄 Processing TSLA...
❌ Required fields not found in cashflow for TSLA
🔄 Processing NVDA...
❌ Required fields not found in cashflow for NVDA
🔄 Processing JNJ...
❌ Required fields not found in cashflow for JNJ
🔄 Processing V...
❌ Required fields not found in cashflow for V
🔄 Processing PG...
❌ Required fields not found in cashflow for PG


ModuleNotFoundError: No module named 'ace_tools'